<a href="https://colab.research.google.com/github/kireetibabu07/SA2025_Capstone_Project/blob/main/My_Final_Capstone_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Inroduction


This sample notebook demonstrates how to process live data streams using Pathway. The dataset used here is a subset of the one provided — specifically, it includes data for only a single parking spot. You are expected to implement your model across all parking spots.

Please note that the pricing model used in this notebook is a simple baseline. You are expected to design and implement a more advanced and effective model.


In [ ]:
!pip install pathway bokeh --quiet # This cell may take a few seconds to execute.

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.6/77.6 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 777.6/777.6 kB 44.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 59.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.3/135.3 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.6/244.6 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.1/319.1 kB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 985.8/985.8 kB 38.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from datetime import datetime
import pathway as pw
import bokeh.plotting
import panel as pn

# Step 1: Importing and Preprocessing the Data

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/dataset.csv')
df

# You can find the sample dataset here: https://drive.google.com/file/d/1D479FLjp9aO3Mg8g6Lpj9oRViWacurA6/view?usp=sharing

,ID,SystemCodeNumber,Capacity,Latitude,Longitude,Occupancy,VehicleType,TrafficConditionNearby,QueueLength,IsSpecialDay,LastUpdatedDate,LastUpdatedTime
0,0,BHMBCCMKT01,577,26.144536,91.736172,61,car,low,1,0,04-10-2016,07:59:00
1,1,BHMBCCMKT01,577,26.144536,91.736172,64,car,low,1,0,04-10-2016,08:25:00
2,2,BHMBCCMKT01,577,26.144536,91.736172,80,car,low,2,0,04-10-2016,08:59:00
3,3,BHMBCCMKT01,577,26.144536,91.736172,107,car,low,2,0,04-10-2016,09:32:00
4,4,BHMBCCMKT01,577,26.144536,91.736172,150,bike,low,2,0,04-10-2016,09:59:00
...,...,...,...,...,...,...,...,...,...,...,...,...
18363,18363,Shopping,1920,26.150504,91.733531,1517,truck,average,6,0,19-12-2016,14:30:00
18364,18364,Shopping,1920,26.150504,91.733531,1487,car,low,3,0,19-12-2016,15:03:00
18365,18365,Shopping,1920,26.150504,91.733531,1432,cycle,low,3,0,19-12-2016,15:29:00
18366,18366,Shopping,1920,26.150504,91.733531,1321,car,low,2,0,19-12-2016,16:03:00


In [ ]:
# Create a Timestamp column
df['Timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'],
                                  format='%d-%m-%Y %H:%M:%S')

In [ ]:
df

,ID,SystemCodeNumber,Capacity,Latitude,Longitude,Occupancy,VehicleType,TrafficConditionNearby,QueueLength,IsSpecialDay,LastUpdatedDate,LastUpdatedTime,Timestamp
0,0,BHMBCCMKT01,577,26.144536,91.736172,61,car,low,1,0,04-10-2016,07:59:00,2016-10-04 07:59:00
1,1,BHMBCCMKT01,577,26.144536,91.736172,64,car,low,1,0,04-10-2016,08:25:00,2016-10-04 08:25:00
2,2,BHMBCCMKT01,577,26.144536,91.736172,80,car,low,2,0,04-10-2016,08:59:00,2016-10-04 08:59:00
3,3,BHMBCCMKT01,577,26.144536,91.736172,107,car,low,2,0,04-10-2016,09:32:00,2016-10-04 09:32:00
4,4,BHMBCCMKT01,577,26.144536,91.736172,150,bike,low,2,0,04-10-2016,09:59:00,2016-10-04 09:59:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
18363,18363,Shopping,1920,26.150504,91.733531,1517,truck,average,6,0,19-12-2016,14:30:00,2016-12-19 14:30:00
18364,18364,Shopping,1920,26.150504,91.733531,1487,car,low,3,0,19-12-2016,15:03:00,2016-12-19 15:03:00
18365,18365,Shopping,1920,26.150504,91.733531,1432,cycle,low,3,0,19-12-2016,15:29:00,2016-12-19 15:29:00
18366,18366,Shopping,1920,26.150504,91.733531,1321,car,low,2,0,19-12-2016,16:03:00,2016-12-19 16:03:00


In [ ]:
# Step 4: Encode Categorical Variables
# List of all expected dummy columns
expected_encoded_columns = [
    'VehicleType_bike', 'VehicleType_truck',
    'TrafficConditionNearby_low', 'TrafficConditionNearby_medium'
]

In [ ]:
# Perform one-hot encoding
df_encoded = pd.get_dummies(df[['VehicleType', 'TrafficConditionNearby']], drop_first=True)


In [ ]:
# Add missing columns manually (fill with 0)
for col in expected_encoded_columns:
    if col not in df_encoded.columns:
        df_encoded[col] = 0

In [ ]:
# Combine Encoded Columns with Main Data
df_model = pd.concat([df, df_encoded], axis=1)

In [ ]:
# Feature Engineering
df_model['OccupancyRate'] = df_model['Occupancy'] / df_model['Capacity']

In [ ]:
# Define features and target
features = [
    'OccupancyRate', 'QueueLength', 'IsSpecialDay',
    'VehicleType_truck', 'VehicleType_bike',
    'TrafficConditionNearby_low', 'TrafficConditionNearby_medium'
]


In [ ]:
# Target (approximate demand)
target = df_model['OccupancyRate'] + df_model['QueueLength'] * 0.05


In [ ]:
# Prepare X and y as float64 explicitly
X = df_model[features].astype(float).values
y = target.astype(float).values.reshape(-1, 1)

{'bias': np.float64(4.774012018080032e-06), 'OccupancyRate': np.float64(0.9999893422320929), 'QueueLength': np.float64(0.05000025193056572), 'IsSpecialDay': np.float64(-2.1009877262806453e-06), 'VehicleType_truck': np.float64(8.788900843497045e-08), 'VehicleType_bike': np.float64(0.0), 'TrafficConditionNearby_low': np.float64(-4.604798911969632e-07), 'TrafficConditionNearby_medium': np.float64(0.0)}


In [ ]:
# Add bias column
X_bias = np.hstack((np.ones((X.shape[0], 1)), X))

In [ ]:
# Ridge Regression
lambda_reg = 0.01
I = np.eye(X_bias.shape[1])
I[0, 0] = 0

In [ ]:
# Solve for weights
weights = np.linalg.inv(X_bias.T @ X_bias + lambda_reg * I) @ X_bias.T @ y
weights = weights.flatten()

In [ ]:
# Display the feature weights
feature_names = ['bias'] + features
feature_weights = dict(zip(feature_names, weights))
print(feature_weights)

{'bias': np.float64(4.774012018080032e-06), 'OccupancyRate': np.float64(0.9999893422320929), 'QueueLength': np.float64(0.05000025193056572), 'IsSpecialDay': np.float64(-2.1009877262806453e-06), 'VehicleType_truck': np.float64(8.788900843497045e-08), 'VehicleType_bike': np.float64(0.0), 'TrafficConditionNearby_low': np.float64(-4.604798911969632e-07), 'TrafficConditionNearby_medium': np.float64(0.0)}


In [ ]:
# Copy from the main model dataframe
df_stream = df_model.copy()

In [ ]:
# Convert LastUpdatedDate + Time into full timestamp
df_stream['Timestamp'] = pd.to_datetime(df_stream['LastUpdatedDate'] + ' ' + df_stream['LastUpdatedTime'],
                                        format='%d-%m-%Y %H:%M:%S')
df_stream = df_stream.sort_values('Timestamp').reset_index(drop=True)


In [ ]:
# Convert all boolean columns in the entire DataFrame
for col in df_stream.columns:
    if df_stream[col].dtype == 'bool':
        df_stream[col] = df_stream[col].astype(int)

In [ ]:
print(df_stream.dtypes)

ID                                        int64
SystemCodeNumber                         object
Capacity                                  int64
Latitude                                float64
Longitude                               float64
Occupancy                                 int64
VehicleType                              object
TrafficConditionNearby                   object
QueueLength                               int64
IsSpecialDay                              int64
LastUpdatedDate                          object
LastUpdatedTime                          object
Timestamp                        datetime64[ns]
VehicleType_car                           int64
VehicleType_cycle                         int64
VehicleType_truck                         int64
TrafficConditionNearby_high               int64
TrafficConditionNearby_low                int64
VehicleType_bike                          int64
TrafficConditionNearby_medium             int64
OccupancyRate                           

In [ ]:
# Select only the required columns to stream
stream_cols = [
    "Timestamp", "Occupancy", "Capacity", "QueueLength", "IsSpecialDay",
    "VehicleType_truck", "VehicleType_bike",
    "TrafficConditionNearby_low", "TrafficConditionNearby_medium"
]

In [ ]:
# Save DataFrame to CSV
df_stream[stream_cols].to_csv("parking_stream.csv", index=False)

In [ ]:
# Pathway schema
class ParkingSchema(pw.Schema):
    Timestamp: str
    Occupancy: int
    Capacity: int
    QueueLength: int
    IsSpecialDay: int
    VehicleType_truck: int
    VehicleType_bike: int
    TrafficConditionNearby_low: int
    TrafficConditionNearby_medium: int

In [ ]:
# Load as stream
data = pw.demo.replay_csv("parking_stream.csv", schema=ParkingSchema, input_rate=1000)

In [ ]:
fmt = "%Y-%m-%d %H:%M:%S"

data = data.with_columns(
    t = data.Timestamp.dt.strptime(fmt),
    day = data.Timestamp.dt.strptime(fmt).dt.strftime("%Y-%m-%dT00:00:00"),
    OccupancyRate = data.Occupancy / data.Capacity
)

In [ ]:
# Injecting your learned weights
feature_weights = {
    "bias": 4.774e-06,
    "OccupancyRate": 0.99999,
    "QueueLength": 0.05000,
    "IsSpecialDay": -2.10e-06,
    "VehicleType_truck": 8.78e-08,
    "VehicleType_bike": 0.0,
    "TrafficConditionNearby_low": -4.60e-07,
    "TrafficConditionNearby_medium": 0.0
}

In [ ]:
# Demand computation using weights
data = data.with_columns(
    demand = (
        feature_weights["bias"]
        + feature_weights["OccupancyRate"] * data.OccupancyRate
        + feature_weights["QueueLength"] * data.QueueLength
        + feature_weights["IsSpecialDay"] * data.IsSpecialDay
        + feature_weights["VehicleType_truck"] * data.VehicleType_truck
        + feature_weights["VehicleType_bike"] * data.VehicleType_bike
        + feature_weights["TrafficConditionNearby_low"] * data.TrafficConditionNearby_low
        + feature_weights["TrafficConditionNearby_medium"] * data.TrafficConditionNearby_medium
    )
)

# Step 2: Making a simple pricing function

In [ ]:
import datetime
import pypw as pw

# Step 3: Use sum/count and manually bound the result using pw.if_else
delta_window = (
    data.windowby(
        pw.this.t,
        instance=pw.this.day,
        window=pw.temporal.tumbling(datetime.timedelta(days=1)),
        behavior=pw.temporal.exactly_once_behavior()
    )
    .reduce(
        t = pw.this._pw_window_end,
        total_demand = pw.reducers.sum(pw.this.demand),
        count = pw.reducers.count()
    )
    .with_columns(
        demand_avg = pw.this.total_demand / pw.this.count,

        # Normalize demand to [-0.5, 0.5]
        # This is the 'bounded_demand_term' from our previous discussion
        normalized_demand_term = pw.if_else(
            pw.this.demand_avg > 1.0,
            0.5,
            pw.if_else(
                pw.this.demand_avg < 0.0,
                -0.5,
                pw.this.demand_avg - 0.5
            )
        ),

        # Calculate the base price before final clamping
        calculated_price_base = 10 * (1 + pw.this.normalized_demand_term),

        # Compute final price using bounded normalized demand, clamping between 5 and 20
        price = pw.if_else(
            pw.this.calculated_price_base > 20,
            20,
            pw.if_else(
                pw.this.calculated_price_base < 5,
                5,
                pw.this.calculated_price_base
            )
        )
    )
)

# Step 3: Visualizing Daily Price Fluctuations with a Bokeh Plot

In [ ]:
pn.extension()

def price_plotter(source):
    fig = bokeh.plotting.figure(
        height=400,
        width=800,
        title="Pathway: Daily Dynamic Parking Price",
        x_axis_type="datetime"
    )
    fig.line("t", "price", source=source, line_width=2, color="navy")
    fig.circle("t", "price", source=source, size=6, color="red")
    return fig

viz = delta_window.plot(price_plotter, sorting_col="t")
pn.Column(viz).servable()


Column
    [0] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)

In [ ]:
# Run the Real-Time Engine
%%capture --no-display
pw.run()


Output()